In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Importacion de librerias

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import tensorflow as tf
import random
import IPython.display as display
from random import randint

# Configuraciones de TensorFlow

### Configuracion de TensorFlow 2.0

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

### Configuracion de TensorFlow 1.0

In [3]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

### Modo eagerly

In [ ]:
#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

# Carga de Datos

In [4]:
root_path = "/home/jefelitman/DataSets/ucf101/split_1"
root_path

'/home/jefelitman/DataSets/ucf101/split_1'

In [ ]:
splits = sorted(os.listdir(root_path))
splits

In [ ]:
modes = ["train","test"]

In [16]:
import sys
sys.path.append('../')

from Datasets_utils.DatasetsLoader import UCF101

dataset = UCF101(root_path,10,True)

In [6]:
batch = dataset.get_next_train_batch([112,112],16)
batch

<tf.Tensor 'packed_10:0' shape=(10, 16, 112, 112, 3) dtype=float32>

1

# Red Neuronal

In [15]:
from LTC_v2 import LTC

Sesion = tf.Session(config=config)

ltc = LTC(101)

In [ ]:
for i in range(dataset.train_batches):
    print("Batch: ",i)
    batch = dataset.get_next_train_batch([112,112],16)
    

In [17]:
Sesion.close()

In [ ]:
from LTC_v2 import LTC

model = LTC(101):

logging_hook = tf.train.LoggingTensorHook(
      tensors={'step': model.global_step,
               'loss': model.cost,
               'precision': precision},
      every_n_iter=100)

training_session = tf.train.MonitoredTrainingSession(
    checkpoint_dir="./Checkpoints",
      hooks=[logging_hook, _LearningRateSetterHook()],
      chief_only_hooks=[summary_hook],
      # Since we provide a SummarySaverHook, we need to disable default
      # SummarySaverHook. To do that we set save_summaries_steps to 0.
      save_summaries_steps=0,
      config=config
)


In [ ]:
https://www.math.purdue.edu/~nwinovic/slides/Getting_Started_with_TensorFlow_II.pdf